In [6]:
#!/usr/bin/env python
# coding: utf-8

# ## Transaction Bundler utility
# 
# - inputs = resource as pyfhir model to append to bundle and existing bundle as pyfhir model to append to.
# 

# In[29]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# This allows you to import the desired function from the module hierarchy:
from pyFHIR_models.fhir_model_generator.model import bundle, organization
from datetime import datetime, date, timedelta
import uuid
import FHIR_templates as f
from json import dumps
from yaml import dump as y_dump

def to_json(r):
    return(dumps(r.as_json(), indent=2))

def to_yaml(r):
    '''serialize pyfhir object to yaml'''
    r_dict = dict(r.as_json())
    return(y_dump(r_dict, sort_keys=False))


def bundle_me(fhir_server, pyfhir_res, fhir_bundle=None):
    file_ts = datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
    if not pyfhir_res.id:
        new_urn = uuid.uuid1().urn # new urn for resource
        pyfhir_res.id = new_urn[9:]
    e = bundle.BundleEntry()
    e.fullUrl = f'{fhir_server}/{pyfhir_res.id}'
    e.resource = pyfhir_res
    e.request = bundle.BundleEntryRequest()
    #e.request.method = 'POST'
    e.request.method = 'PUT'
    e.request.url = f'{pyfhir_res.resource_type}/{pyfhir_res.id}'
    if fhir_bundle: #add entry
        pass
    else:  # create transaction bundle
        bundle_type = 'transaction'
        bundle_id = f'transaction-bundle-{file_ts}'   
        fhir_bundle = bundle.Bundle(
            dict(
                id = bundle_id,
                type = bundle_type,
                timestamp = datetime.now().isoformat(),
                entry = [],
            )
        )
    fhir_bundle.entry.append(e)
    return(fhir_bundle)


# In[43]:


test_server = 'http://test.fhir.org/r4'
org = organization.Organization(f.example_org)
tbundle = bundle_me(test_server,org)
print(dumps(tbundle.as_json(),indent=2))
print(to_json(org))
print(to_yaml(org))









{
  "resourceType": "Bundle",
  "id": "transaction-bundle-20200923165124228315",
  "type": "transaction",
  "timestamp": "2020-09-23T09:51:24.228407",
  "entry": [
    {
      "fullUrl": "http://test.fhir.org/r4/example-organization-2",
      "resource": {
        "resourceType": "Organization",
        "id": "example-organization-2",
        "meta": {
          "profile": [
            "http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization"
          ]
        },
        "identifier": [
          {
            "system": "http://hl7.org.fhir/sid/us-npi",
            "value": "1407071236"
          },
          {
            "system": "http://example.org/fhir/sid/us-tin",
            "value": "121111111"
          }
        ],
        "active": true,
        "type": [
          {
            "coding": [
              {
                "system": "http://terminology.hl7.org/CodeSystem/organization-type",
                "code": "prov",
                "display": "Healthcare